In [ ]:
let buildDir = "./bin/Debug/net8.0" 
let copyDir = "./bin/Debug/notebook"

// Clear existing copy directory
if System.IO.Directory.Exists(copyDir) then 
    System.IO.Directory.Delete(copyDir, true)

let rec copyBuildDir (source : string) (target: string) =
    System.IO.Directory.CreateDirectory(target) |> ignore  
    for file in System.IO.Directory.GetFiles(source) do 
        File.Copy(file, System.IO.Path.Combine(target, System.IO.Path.GetFileName(file)))

    for dir in System.IO.Directory.GetDirectories(source) do
        let newDir = System.IO.Path.Combine(target, System.IO.Path.GetFileName(dir)) 
        copyBuildDir dir newDir

copyBuildDir buildDir copyDir

In [ ]:
#r "./bin/Debug/notebook/studies.dll"
#r "./bin/Debug/notebook/core.fs.dll"

#r "nuget:FSharp.Data"
#r "nuget: Plotly.NET.Interactive"

In [ ]:
// read params from files

let filterSelectionString = System.IO.File.ReadAllText("filter.txt")
let filterDirectionString = System.IO.File.ReadAllText("filter_direction.txt")
let resultsDirectory = System.IO.File.ReadAllText("results_directory.txt")
let studiesDirectory = System.IO.File.ReadAllText("study_directory.txt")
let inputFilename    = $"{studiesDirectory}\\signals.csv"

System.Console.WriteLine("Study directory: " + studiesDirectory)
System.Console.WriteLine("Input filename: " + inputFilename)
System.Console.WriteLine("Results directory: " + resultsDirectory)
System.Console.WriteLine("Filter Selection: " + filterSelectionString)
System.Console.WriteLine("Filter Direction: " + filterDirectionString)

In [ ]:
open FSharp.Data
open Plotly.NET
open core.fs.Stocks

let outputToConsole (message:string) =
    System.Console.WriteLine(message)
        
// functions that deal with outcomes and turn them into summaries, describes them, etc
let columnChart label keys values =
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
let histogramChart label bins values =
    let chart = 
        Chart.Histogram(
            X = values,
            NBinsX = bins,
            Name = label
        )
    chart.Display() |> ignore

let outputSummary outputFunc includeDistributionChart (summary:studies.Trading.TradeSummary) = 

    // this ensures that the numbers align instead of negative EV taking up more space
    let evPositiveSign =
        match summary.EV with
        | x when x >= 0m -> "+"
        | _ -> ""
    
    let format = "0.00"
    let percentFormat = "00.00%"
    outputFunc $"{evPositiveSign}{summary.EV.ToString(percentFormat)} EV -> {summary.WinPct.ToString(percentFormat)} \t {summary.AvgWin.ToString(percentFormat)} \t {summary.AvgLoss.ToString(percentFormat)} \t {summary.AvgGainLoss.ToString(format)} avg gain/loss \t {summary.TotalGain.ToString(format)} total gain  \t [{summary.NumberOfTrades} trades] \t {summary.StrategyName}"
        
    match includeDistributionChart with
    | true ->
        histogramChart "Gain Distribution" 40 (summary.Gains)
    | false -> ()
    
let screenerIdOptionEquals value opt =
    opt |> Option.defaultValue 0 = value
    
let greaterThan value opt =
    opt |> Option.defaultValue 0m > value
    
let lessThan value opt =
    opt |> Option.defaultValue 0m < value
    
let genericFilters = [
    ("All", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> true)
    ("New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> screenerIdOptionEquals 28)
    ("Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid  |> screenerIdOptionEquals 29)
    ("Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid  |> screenerIdOptionEquals 30)
    ("New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid  |> screenerIdOptionEquals 31)
]

let gapUpFilters = [
    ("Gap ups - New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Gap |> greaterThan 0m && o.Screenerid |> screenerIdOptionEquals 28)
    ("Gap ups - Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Gap |> greaterThan 0m && o.Screenerid |> screenerIdOptionEquals 29)
    ("Gap downs - Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Gap |> lessThan 0m && o.Screenerid |> screenerIdOptionEquals 30)
    ("Gap downs - New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Gap |> lessThan 0m && o.Screenerid |> screenerIdOptionEquals 31)
]

let sma20PriceFilters = [
    ("price>20 - New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 28)
    ("price>20 - Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 29)
    ("price>20 - Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 30)
    ("price>20 - New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 31)

    ("price<20 - New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 28)
    ("price<20 - Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 29)
    ("price<20 - Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 30)
    ("price<20 - New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 31)
]

let sma200PriceFilters = [
    ("price>200 - New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 28)
    ("price>200 - Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 29)
    ("price>200 - Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 30)
    ("price>200 - New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> lessThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 31)

    ("price<200 - New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 28)
    ("price<200 - Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 29)
    ("price<200 - Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 30)
    ("price<200 - New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma200 |> greaterThan o.OpenPrice && o.Screenerid |> screenerIdOptionEquals 31)
    
    
    ("price>smaAlign - New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid |> screenerIdOptionEquals 28)
    ("price>smaAlign - Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid |> screenerIdOptionEquals 29)
    ("price>smaAlign - Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid |> screenerIdOptionEquals 30)
    ("price>smaAlign - New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> lessThan o.OpenPrice && o.Sma20 > o.Sma50 && o.Sma50 > o.Sma150 && o.Sma150 > o.Sma200 && o.Screenerid |> screenerIdOptionEquals 31)

    ("price<smaAlign - New Highs", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid |> screenerIdOptionEquals 28)
    ("price<smaAlign - Top Gainers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid |> screenerIdOptionEquals 29)
    ("price<smaAlign - Top Losers", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid |> screenerIdOptionEquals 30)
    ("price<smaAlign - New Lows", fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Sma20 |> greaterThan o.OpenPrice && o.Sma20 < o.Sma50 && o.Sma50 < o.Sma150 && o.Sma150 < o.Sma200 && o.Screenerid |> screenerIdOptionEquals 31)
]

// filter out outcomes that have minimum of trades we are interested in
let minimumTradesPerCategory = 10

[<Flags>]
type FilterSelection =
    | GenericFilters = 1
    | Gaps = 2
    | Sma20 = 4
    | Sma200 = 8
    | All = 255

let createTradeSummaries filterSelection outcomes =
    
    let tradesGroupedByStrategy =
        outcomes
        |> Seq.groupBy (fun (t:studies.Trading.TradeOutcomeOutput.Row) -> t.Strategy) // all strategies, no filters

    let filterNamePairs = [
        if filterSelection &&& FilterSelection.GenericFilters = FilterSelection.GenericFilters then yield! genericFilters    
        if filterSelection &&& FilterSelection.Gaps = FilterSelection.Gaps then yield! gapUpFilters
        if filterSelection &&& FilterSelection.Sma20 = FilterSelection.Sma20 then yield! sma20PriceFilters
        if filterSelection &&& FilterSelection.Sma200 = FilterSelection.Sma200 then yield! sma200PriceFilters
    ]

    filterNamePairs
    |> Seq.collect (
        fun (filterName,filter) ->

            tradesGroupedByStrategy
            |> Seq.map (
                fun (strategy, trades) ->
                    let filteredTrades = trades |> Seq.filter filter
                    
                    let strategy = filteredTrades |> Seq.tryHead |> Option.map (fun x -> x.Strategy)
                    match strategy with
                    | None -> None
                    | Some strategy ->
                        let name = $"{filterName}, {strategy}"
                        let summary = filteredTrades |> studies.Trading.TradeSummary.create name
                        match summary.NumberOfTrades with
                        | x when x < minimumTradesPerCategory -> None
                        | _ ->
                            let invalidCharacters = System.IO.Path.GetInvalidFileNameChars()
                            let cleanedFilename = invalidCharacters |> Seq.fold (fun (n:string) (c:char) -> n.Replace(c, '_')) name
                            let finalFilename = $"{filterSelectionString}_{filterDirectionString}\\{cleanedFilename}.csv"
                            let rows  = new studies.Trading.TradeOutcomeOutput(filteredTrades)
                            let finalPath = System.IO.Path.Combine(resultsDirectory,finalFilename)
                            
                            rows.Save(finalPath)
                            Some summary
            )
            |> Seq.choose id
    )
    

type OutcomeGrouping =
    | Signal
    | Strategy
    
let describeOutcomes outputFileOption outcomeGrouping includeEVChart includeDistributionChart tradeSummaries =
    
    let outputContent message =
        match outputFileOption with
        | None -> ()
        | Some outputFile -> System.IO.File.AppendAllText(outputFile, message + System.Environment.NewLine)
        System.Console.WriteLine(message)
        
    let groupingFunction =
        match outcomeGrouping with
        | OutcomeGrouping.Signal -> (fun (s:studies.Trading.TradeSummary) -> s.StrategyName.Split(",")[0])
        | OutcomeGrouping.Strategy -> (fun (s:studies.Trading.TradeSummary) -> s.StrategyName.Split(",")[1])
        
    let groupedData = tradeSummaries |> Seq.groupBy groupingFunction
    
    groupedData
    |> Seq.iter (
        fun (groupingName,summaries) ->
            
            outputContent $"{groupingName}"
            
            let sortedSummaries = summaries |> Seq.sortByDescending (fun summary -> summary.EV)
            
            sortedSummaries |> Seq.iter (fun summary -> outputSummary outputContent includeDistributionChart summary)
            
            if includeEVChart then
                let keys = sortedSummaries |> Seq.map (fun summary -> summary.StrategyName)
                let values = sortedSummaries |> Seq.map (fun summary -> summary.EV)
            
                columnChart groupingName keys values
                
            outputContent ""
            outputContent ""
    )
    
let highlightStrategies numberOfTopRecords tradeSummaries =
        
    let sortFunctions = [
        ("EV", fun (t:studies.Trading.TradeSummary) -> t.EV)
        //("Total Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.TotalGain)
        //("Win %", fun (t:studies.ScreenerStudy.TradeSummary) -> t.WinPct)
        //("Avg Gain", fun (t:studies.ScreenerStudy.TradeSummary) -> t.AvgGainLoss)
    ]

    sortFunctions
    |> Seq.iter (fun (sortLabel, sortFunction) ->
        printfn $"Top {numberOfTopRecords} by: {sortLabel}"
        
        let summaryFunc = outputSummary outputToConsole false
        let descending =
            tradeSummaries
            |> Seq.sortByDescending sortFunction
            |> Seq.truncate numberOfTopRecords
            |> Seq.iter summaryFunc
        
        outputToConsole "..."
        
        let ascending =
            tradeSummaries
            |> Seq.sortBy sortFunction
            |> Seq.truncate numberOfTopRecords
            |> Seq.sortByDescending sortFunction
            |> Seq.iter summaryFunc
            
        printfn ""
        printfn ""
    )

In [ ]:
let noFilter = 
    let filter (row:studies.DataHelpers.ISignal) = true
    "NO filter", filter
    
let myCycleFilter  =
    let filter (row:studies.DataHelpers.ISignal) = 
        match row.Date with
        | x when x >= "2022-04-22" && x <= "2022-05-25" -> true
        | x when x >= "2022-06-10" && x <= "2022-07-07" -> true
        | x when x >= "2022-08-19" && x <= "2022-10-17" -> true
        | x when x >= "2022-12-20" && x <= "2023-01-06" -> true
        | x when x >= "2023-02-16" && x <= "2023-03-28" -> true
        | x when x >= "2023-05-02" && x <= "2023-06-02" -> true
        | x when x >= "2023-08-02" && x <= "2023-11-02" -> true
        | x when x >= "2024-01-05" && x <= "2024-01-16" -> true
        | x when x >= "2024-04-04" && x <= "2024-04-16" -> true
        | _ -> false
    "My Cycle DOWN", filter

let spyShortTermFilter =
    let filter (row:studies.DataHelpers.ISignal) = 
        match row.Date with
        | x when x >= "2022-04-26" && x <= "2022-07-28" -> true
        | x when x >= "2022-09-14" && x <= "2022-11-09" -> true
        | x when x >= "2022-12-27" && x <= "2023-01-25" -> true
        | x when x >= "2023-03-10" && x <= "2023-04-10" -> true
        | x when x >= "2023-08-22" && x <= "2023-11-14" -> true
        | x when x >= "2024-04-19" && x <= "2024-05-13" -> true
        | _ -> false
    "SPY short-term DOWN", filter

let spyLongTermDown  =
    let filter (row:studies.DataHelpers.ISignal) = 
        match row.Date with
        | x when x >= "2022-03-11" && x <= "2023-01-25" -> true
        | _ -> false
    "SPY long-term DOWN", filter
    
// use this cell for signal filtering
type CycleDirection =
    | Up
    | Down
    | All
    
    with 
        static member FromString str =
            match str with
            | nameof Up -> Up
            | nameof Down -> Down
            | nameof All -> All
            | _ -> failwith $"Unexpected Cycle Direction value: {str}" 
    
type FilterSelection =
    | NoFilter
    | MyCycle
    | SpyShortTermCycle
    | SpyLongTermCycle
    
    with
        static member FromString str =
            match str with
            | nameof NoFilter -> NoFilter
            | nameof MyCycle -> MyCycle
            | nameof SpyShortTermCycle -> SpyShortTermCycle
            | nameof SpyLongTermCycle -> SpyLongTermCycle
            | _ -> failwith $"Unexpected filter selection value: {str}"
            
let getFilter selection =
    match selection with
    | NoFilter -> noFilter
    | MyCycle -> myCycleFilter
    | SpyShortTermCycle -> spyShortTermFilter
    | SpyLongTermCycle -> spyLongTermDown

let selection = filterSelectionString |> FilterSelection.FromString
let direction = filterDirectionString |> CycleDirection.FromString

let cycleLabel, cycleFilter = selection |> getFilter

let filter (row:studies.DataHelpers.ISignal) = 
    match direction with
    | CycleDirection.Down -> row |> cycleFilter
    | CycleDirection.Up -> row |> cycleFilter |> not
    | CycleDirection.All -> true    

In [ ]:
let inputSignals =
    inputFilename
    |> studies.BreakoutStudy.BreakoutSignalRecord.Load
    |> _.Rows
    |> Seq.map studies.BreakoutStudy.BreakoutSignalRecordWrapper
    |> Seq.cast<studies.DataHelpers.ISignal>
    |> Seq.toList
    
outputToConsole $"Loaded {inputSignals.Length} signals"

outputToConsole $"SELECTED cycle: {cycleLabel}, taking {direction} signals"
outputToConsole $"filtering out inputs..."

let signals = inputSignals |> List.filter filter

outputToConsole $"after filtering, we have {signals.Length} signals"

let priceFunc ticker = studies.DataHelpers.getPricesFromCsv studiesDirectory ticker

let strategies = [
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long None None   // no stop, buy and hold or sell and hold
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short None None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 5) None // no stop, but only hold for 30 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 5) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 10) None // no stop, but only hold for 30 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 10) None    
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 30) None // no stop, but only hold for 30 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 30) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 60) None // no stop, but only hold for 60 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 60) None
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long None (Some 0.05m) // 5% stop
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short None (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 30) (Some 0.05m) // 5% stop plus holding at most 30 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 30) (Some 0.05m)
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Long (Some 60) (Some 0.05m) // 5% stop plus holding at most 60 bars
    studies.Trading.strategyWithStopLossPercent false StockPositionType.Short (Some 60) (Some 0.05m)
    studies.Trading.strategyWithSignalOpenAsStop false  // signal open as stop
    studies.Trading.strategyWithSignalCloseAsStop false // signal close as stop
    studies.Trading.strategyWithTrailingStop false StockPositionType.Long 0.1m  // trailing stop of 10%
    studies.Trading.strategyWithTrailingStop false StockPositionType.Short 0.1m
]

let outcomes = 
    strategies
    |> studies.Trading.runTrades priceFunc signals
    |> Async.RunSynchronously
    |> Seq.toList

printfn "Finished running trades, generated:"
printfn $"{outcomes.Length} trade outcomes"

In [ ]:
// leave this cell for outcome checking from time to time when troubleshooting
//outcomes |> List.filter (fun (o:studies.ScreenerStudy.TradeOutcomeOutput.Row) -> o.Screenerid = 28 && o.OpenPrice < o.Sma200 && o.Strategy = "Buy")

In [ ]:
printfn "Generating trade summaries..."
let summaryBreakdown = FilterSelection.All
let tradeSummaries = outcomes |> createTradeSummaries summaryBreakdown |> Seq.toList
printfn $"{tradeSummaries.Length} summaries generated"

In [ ]:
"Highlighting trades" |> outputToConsole

tradeSummaries
|> highlightStrategies 20

In [ ]:
tradeSummaries
|> describeOutcomes None OutcomeGrouping.Strategy false false

printfn "Finished"

In [ ]:
let exploreCertainStrategies() =
    let strategiesOfInterest = [
        "SL of"
        "as stop"
    ]

    let summarizeSubset (name:string) =
        tradeSummaries
            |> Seq.filter (fun summary -> summary.StrategyName.Contains(name))
            |> Seq.sortByDescending (fun summary -> summary.EV)
            |> Seq.iter (fun summary -> 
                outputSummary outputToConsole false summary
            )

        printfn ""

    strategiesOfInterest |> List.iter (fun n -> summarizeSubset n)

// exploreCertainStrategies()

In [ ]:
// sanity check, print random trades, look over them, investigate specific trades if needed
let positionDescription (signal:studies.Trading.TradeOutcomeOutput.Row) =
    // it will consist of components that describe signal
    // gap up or gap down
    // price>20, price<20, put together into a string

    let signalDescription =
        match signal.Screenerid |> Option.defaultValue 0 with
        | 28 -> "New High"
        | 29 -> "Top Gainer"
        | 30 -> "Top Loser"
        | 31 -> "New Low"
        | _ -> "Generic Signal"
    
    let gapDescription =
        match signal.Gap |> Option.defaultValue 0m with
        | x when x > 0m -> "Gap Up"
        | x when x < 0m -> "Gap Down"
        | _ -> ""
        
    let priceDescription =
        match signal.OpenPrice with
        | x when x >= (signal.Sma20 |> Option.defaultValue 0m) -> "price>20"
        | x when x < (signal.Sma20 |> Option.defaultValue 0m) -> "price<20"
        | _ -> failwith "freakout"
        
    String.concat " " [signalDescription; gapDescription; priceDescription]
        
let outcomeDescription (x:studies.Trading.TradeOutcomeOutput.Row) =
    let gain = x.PercentGain.ToString("##.##%")
    let signal = x |> positionDescription
    printfn $"{x.Ticker} {x.Strategy}: {x.Opened} -> {x.Closed}, {x.OpenPrice} -> {x.ClosePrice}: profit of {gain}, signal: {signal}, strat: {x.Strategy}"
    
let random = System.Random()

let tradeStrategyOfInterest = "Sell"

//let filterFunc x = x |> signalDescription |> _.Contains(tradeStrategyOfInterest)
let filterFunc (x:studies.Trading.TradeOutcomeOutput.Row) = x.Strategy = tradeStrategyOfInterest

outcomes
|> Seq.filter filterFunc
|> Seq.sortBy (fun x -> random.NextInt64())
|> Seq.take 10
|> Seq.iter outcomeDescription

printfn ""
printfn $"Max gain trade from {tradeStrategyOfInterest}"

outcomes
|> Seq.filter filterFunc
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

printfn ""
printfn "Max gain overall"

outcomes
|> Seq.maxBy (fun x -> x.PercentGain)
|> outcomeDescription

In [ ]:
Plotly.NET.Defaults.DefaultWidth <- 900

let metrics = [
    ("EV", fun (s:studies.Trading.TradeSummary) -> s.EV)
    ("Win/Loss", fun s -> s.AvgGainLoss)
    ("Win %", fun s -> s.WinPct)
    ("Avg Win", fun s -> s.AvgWin)
    ("Avg Loss", fun s -> s.AvgLoss)
    ("Number of Trades", fun s -> s.NumberOfTrades)
]

metrics
|> List.iter( fun (label,metricFunc) ->

    outputToConsole $"sorting for {label}..."
    
    let sorted = tradeSummaries |> Seq.sortByDescending metricFunc |> Seq.toList
    
    let values = sorted |> List.map metricFunc
    let keys = sorted |> List.map (fun summary -> summary.StrategyName)

    outputToConsole $"mapped keys and values for {label}, charting..."
    
    // values |> List.zip keys |> List.iter (fun (l,v) -> outputToConsole $"{l}: {v}")
    
    let column = 
        Chart.Column(values = values, Keys = keys)
        |> Chart.withYAxisStyle (TitleText = label)
    
    column.Display() |> ignore
    
    outputToConsole ""
)

In [ ]:
// let's take only the trades where EV is positive
outputToConsole "Positive EVs only, sorted by EV descending"

tradeSummaries
|> List.filter (fun s -> s.EV > 0m)
|> List.sortByDescending (fun s -> s.EV)
|> List.iter (fun s -> outputSummary outputToConsole false s)

In [ ]:
let benchmarkStrategyName = "All, Sell"

//tradeSummaries |> List.map _.StrategyName |> List.distinct |> List.iter outputToConsole
let benchmarkStrategy = tradeSummaries |> List.find (fun s -> s.StrategyName = benchmarkStrategyName)

outputSummary outputToConsole false benchmarkStrategy

In [ ]:
let newHighsWithPriceBelow20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 28 && o.OpenPrice < (o.Sma20 |> Option.defaultValue 0m) )

let count = newHighsWithPriceBelow20 |> Seq.length

//printfn "Found %i" count
//newHighsWithPriceBelow20 |> Seq.take 10 |> Seq.iter outcomeDescription

In [ ]:
let topLoserPriceAbove20 = outcomes |> Seq.filter (fun (o:studies.Trading.TradeOutcomeOutput.Row) -> o.Screenerid |> Option.defaultValue 0 = 30 && o.OpenPrice > (o.Sma20 |> Option.defaultValue 0m) && o.Strategy = "Sell")

let count = topLoserPriceAbove20 |> Seq.length

//printfn "Found %i" count
//topLoserPriceAbove20 |> Seq.take 30 |> Seq.iter outcomeDescription